<a href="https://colab.research.google.com/github/munendra-kondeti/torrent-downloader/blob/main/torrentClient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install requirements

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

#Import

In [ ]:
import libtorrent as lt
import os
import time

#Upload Torrent file into Colab

In [ ]:
import os
from google.colab import files

# Create a new directory named 'temp_dir'
os.mkdir('temp_dir')

# Change the current working directory to 'temp_dir'
os.chdir('temp_dir')

# Upload files to the 'temp_dir' directory
uploaded = files.upload()

for fn in uploaded.keys():
 print('User uploaded file "{name}" with length {length} bytes'.format(
    name=fn, length=len(uploaded[fn])))


In [ ]:
file = "/content/temp_dir/sintel.torrent"

#Torrent magnet link set here

In [ ]:
link = "magnet:?xt=urn:btih:08ada5a7a6183aae1e09d831df6748d566095a10&dn=Sintel&tr=udp%3A%2F%2Fexplodie.org%3A6969&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969&tr=udp%3A%2F%2Ftracker.empire-js.us%3A1337&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337&tr=wss%3A%2F%2Ftracker.btorrent.xyz&tr=wss%3A%2F%2Ftracker.fastcast.nz&tr=wss%3A%2F%2Ftracker.openwebtorrent.com&ws=https%3A%2F%2Fwebtorrent.io%2Ftorrents%2F&xs=https%3A%2F%2Fwebtorrent.io%2Ftorrents%2Fsintel.torrent"

#1->Torrent session
<p>create_session </p>
<p>show_ports </p>
<p>update_ports </p>

In [ ]:
class TorrentSession:
  ''' TorrentSession Handles all the session related'''

  # listing ports range Default
  PORTS = {"start": 6881, "end": 6891}

  @staticmethod
  def create_session():
    '''create a session on PORTS = {"start": 6881, "end": 6891} '''
    PORTS = TorrentSession.PORTS
    print("Insiliziating Session")
    session = lt.session()
    session.listen_on(PORTS.get("start"), PORTS.get("end"))
    print("Session created and listing on ports => ", PORTS)
    return session

  @staticmethod
  def show_ports():
    '''Prints the port details. Change ports if need with
    `TorrentSession.PORTS.update` '''
    print(TorrentSession.PORTS)

  @staticmethod
  def update_ports(start : int , end : int) -> None:
    TorrentSession.PORTS.update(start=start,end=end)
    print("done")


<h2> For Testing </h2>

In [ ]:
# update ports
# TorrentSession.update_ports(start=6882,end=6888)
# Create a session
# TorrentSession.create_session()
# For checking ports
# TorrentSession.show_ports()

#Torrent file handler
<h2>TorrentFile</h2>
<p> is_torrent_file </p>
<p> is_magnet_link </p>
<p> file_size </p>

In [ ]:
class TorrentFile:
  '''It is the class that handles the `Torrent Files related`'''
  def __init__(self) -> None:
    pass

  def is_torrent_file(self,file:str) -> bool:
    '''Takes the file location as `InPut`.
     Checks the file end with `Torrent` or not'''
    return file.endswith('.torrent')

  def is_magnet_link(self,link:str) -> bool:
    '''Checks the link is it `Magnet` or not'''
    return link.startswith('magnet:?')

  def file_size(self,bytes) -> dict:
    ''' Converts `bytes` to the respective `unites` Ex:-{byte,kb,mb,gb,tb}'''
    byte = bytes
    sizes = {
        "byte":0,
        "kb" : 0,
        "mb" : 0,
        "gb" : 0,
        "tb" : 0
    }
    byte_fixed_size = 1000 # in general 1024 but in python 1000
    kb = byte / byte_fixed_size
    sizes.update(kb=kb)
    sizes.update(byte=0)
    if sizes.get('kb') >= byte_fixed_size:
      mb = byte/(byte_fixed_size ** 2)
      sizes.update(mb=mb)
      sizes.update(kb=0)
    if sizes.get('mb') >= byte_fixed_size:
      gb = byte/(byte_fixed_size ** 3)
      sizes.update(gb=gb)
      sizes.update(mb=0)
    if sizes.get('gb') >= byte_fixed_size:
      tb = byte/(byte_fixed_size** 4)
      sizes.update(tb=tb)
      sizes.update(gb=0)

    units = sizes
    # Distinguish Valid unit
    temp={}
    for unit in units:
      if units.get(unit) != 0 :
        temp = {unit:units.get(unit)}
    return temp



<h2>Testing </h2>

In [ ]:
# tf = TorrentFile()
# tf.is_magnet_link('magnet:?')
# tf.is_torrent_file('.torrent')
# tf.file_size(1025)

#Download Class

In [ ]:
class Download(TorrentFile):
  '''It handles all the downloads related'''

  def __init__(self,session:object, save_path = None) ->None:
    '''Give the `session` as input `save_path` is default temp location'''
    self.save_path = save_path if save_path is not None else (os.getcwd()+"/temp/")
    self.session = session

  def add_multiple(self,file_or_Link:str) :
    '''Add the files or links to the on going list'''
    self.file_or_Link=file_or_Link
    self.downloads = []
    if self.is_magnet_link(self.file_or_Link):
      # if its a magnet link
      parms = {"save_path": self.save_path}
      torrent_handle = lt.add_magnet_uri(self.session,self.file_or_Link, parms)
      self.downloads.append(torrent_handle)
      print("added magnet link successfully")
      # else if torrent file
    elif self.is_torrent_file(file_or_Link):
      torrentFileInfo = lt.torrent_info(self.file_or_Link)
      parms= {'ti': torrentFileInfo, 'save_path': self.save_path}
      self.downloads.append(self.session.add_torrent(parms))
      print("torrent added successfully")
      # else invalid file
    else :
      print("invalid file or link")

    return self

  def add(self,file_or_Link) -> str :
    '''Add the files or links to the on going list'''
    self.file_or_Link=file_or_Link
    if self.is_magnet_link(self.file_or_Link):
      # if its a magnet link
      parms = {"save_path": self.save_path}
      self.torrent_link = lt.add_magnet_uri(self.session,self.file_or_Link, parms)
      print("added magnet link successfully")
      # else if torrent file
    elif self.is_torrent_file(file_or_Link):
      torrentFileInfo = lt.torrent_info(self.file_or_Link)
      parms= {'ti': torrentFileInfo, 'save_path': self.save_path}
      self.torrent_link=self.session.add_torrent(parms)
      print("torrent added successfully")
      # else invalid file
    else :
      print("invalid file or link")
    start = self.start
    return self

  def start(self,logs=False) -> bool :
    try:
      download = self.torrent_link
      status_method = download.status
      is_seed_method = download.is_seed
      print("Saving downloads @ ",self.save_path
            ,"\n starting download ")
      while not is_seed_method():
          status = status_method()
          if status.num_peers < 1:
              count = 0
              while status.num_peers < 1:
                  if count > 5:
                      break
                  time.sleep(1)
                  count += 1

          total_size = download.get_torrent_info().total_size()
          remaining_bytes = status.total_wanted - status.total_wanted_done if total_size is not None else None
          time.sleep(1)
          if logs:
            print("state: ", status.state,
                  "\ntotal size: ",self.file_size(total_size),
                  "\nRemaing bytes: ", self.file_size(remaining_bytes))

      else:
          print(" Download done")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally :
      print("DONE")
      self.session.remove_torrent(download)


  def start_multiple_downloads(self,logs=False):
    print("Saving downloads @ ",self.save_path)
    while self.downloads:
      for download in self.downloads:
        if not download.is_seed():
          self.status = download.status()
          if self.status.num_peers < 1:
            count = 0
            while self.status.num_peers < 1 :
              time.sleep(1)
              if count > 6 :
                break
              count+=1
          torrent_info = download.get_torrent_info()
          self.total_size = torrent_info.total_size()
          self.remaining_bytes = self.status.total_wanted - self.status.total_wanted_done if torrent_info is not None else None

          if logs:
            print("Downloading ",download.name())
            print("progress: %.2f%%" % (self.status.progress * 100))
            print("download rate: %.2f kB/s" % (self.status.download_rate / 1000))
            print("upload rate: %.2f kB/s" % (self.status.upload_rate / 1000))
            print("num peers: ", self.status.num_peers)
            print("state: ", self.status.state)
            print("total size: ",self.file_size(self.total_size))
            print("Remaing size: ",self.file_size(self.remaining_bytes))
          else:
            print("download done")
        else:
            print("Download Successful @"+self.save_path +" => ",download.name())
            self.session.remove_torrent(download)
            self.downloads.remove(download)

    time.sleep(.5)


  # Download Testing

In [ ]:
ts = TorrentSession.create_session()


In [ ]:
td = Download(ts)

In [ ]:
td.add(link).start()